## Map of data avilability in the German Protest Registrations Dataset

In [1]:
%load_ext autoreload
%autoreload complete

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from matplotlib_inline.backend_inline import set_matplotlib_formats
from tqdm.notebook import tqdm

from protest_impact.util import cache, project_root

set_matplotlib_formats("svg")

See [`data_description.ipynb`](data_description.ipynb) for the code that generates the underlying data.

In [2]:
path = project_root / "protest_impact/data/protests/german_protest_registrations"

with open(path / "gpreg-overview.csv") as f:
    df = pd.read_csv(f)

__Color scheme:__

- blue: good data available
  - \#registrations present
  - data from the regional capital available
- gray-blue: some data available
- gray: no data available

In [3]:
# | label: data-official-map
# | fig-cap: Map of official data. Blue = data including the number of registered protesters is available for the political capital of the region; gray-blue = some other data is available, e. g. without protester numbers or only for other cities than the capital; gray = no data is available or the data is so unstructured that it is not included in the dataset.
# | column: margin

import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats("svg")

# load administrative boundaries

gdf = gpd.read_file(project_root / "data/diva-gis" / "DEU_adm1.shp")
gdf = gdf.to_crs("EPSG:4326")

# color:


def has_data(x):
    return df[(df["region"] == x) & (df["2022"] > 0)].size > 0


def has_good_data(x):
    return (
        df[
            (df["region"] == x)
            & (df["2022"] > 0)
            & (df["capital"] == True)
            & (df["registrations"] == True)
        ].size
        > 0
    )
    return any(
        [
            a
            for a in df.iterrows()
            if a[1]["region"] == x
            and df.loc[a, "cap?"] == "✓"
            and df.loc[a, "#reg?"] == "✓"
        ]
    )


gdf["color"] = gdf["NAME_1"].apply(
    lambda x: "blue"
    if has_data(x) and has_good_data(x)
    else ("steelblue" if has_data(x) else "lightgray")
)
gdf["color"] = gdf["color"].astype("category")
gdf["color"] = gdf["color"].cat.set_categories(["lightgray", "steelblue", "blue"])

# plot:

fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, color=gdf["color"], edgecolor="black", linewidth=0.5)
ax.set_axis_off()
legend_elements = [
    Patch(facecolor="lightgray", edgecolor="black", label="No data"),
    Patch(facecolor="steelblue", edgecolor="black", label="Some data"),
    Patch(facecolor="blue", edgecolor="black", label="Good data"),
]
ax.legend(
    handles=legend_elements,
    loc="lower center",
    ncol=3,
    bbox_to_anchor=(0.5, -0.05),
    frameon=False,
    fontsize=14,
)
plt.show()